Clean and Merge Data
======================
This notebook loads and combines 3 different data files into a single dataframe. 
It combines _school demographic data_, _math test scortes_, and _ela test scores_.

For this example we try to use more re-usable functions to help clean our data.
In order to **merge** data sets, we need to have "keys" or "indices" in both
dataframes in order to know which rows to merge together. We know that our
schools have a unique key called `dbn` which we can use, but we also need to
combine it with the academic year in order to get a the correct unique id for each row.

- how to join datasets on columns
- how to rename columns


In [1]:
# load the demographic dataimport pandas as pd
import pandas as pd

demo_url = "https://data.cityofnewyork.us/resource/vmmu-wj3w.csv?$limit=1000000"
math_url = "https://data.cityofnewyork.us/resource/kha6-7i9i.csv?$limit=1000000"
ela_url = "https://data.cityofnewyork.us/resource/gu76-8i7h.csv?$limit=1000000"

In [2]:
# load and prep the school demographics data
# generic function to take percents as Strings and convert to real
# expects data to look like '84.33%', 'Above 95%', 'Below 5%'
def str_pct(row, pct_col, enroll_col):
    pct = row[pct_col][:-1]
    # just call the population size `n`
    n = row[enroll_col]
    try:
        pct = float(pct) / 100
    except:
        if "Above" in pct:
            pct = n * .96 / n
        elif "Below" in pct:
            pct = n * .04 / n
    return float(pct)


# because `str_pct` is a generic function, we need to wrap it
# with another function, we can do this with `def` or `lambda`
# we'll do both, here
def pct_pov(row): return str_pct(row, "poverty_1", "total_enrollment")


# add district and boro info
boros = {"K":"Brooklyn", "X":"Bronx", "M": "Manhattan", "Q": "Queens", "R": "Staten Island"}

def district(dbn): return int(dbn[:2])
def boro(dbn): return boros[dbn[2]]

# convert the string format `year` to match the int format academic year
# used in the test score data
def ay(year): return int(year.split("-")[0])


# get the data and clean it a little bit
df = pd.read_csv(demo_url)

df["year"] = df["year"].apply(ay)
df["district"] = df["dbn"].apply(district)
df["boro"] = df["dbn"].apply(boro)
df["poverty_1"] = df.apply(pct_pov, axis = 1)
df["economic_need_index"] = df.apply(lambda row: str_pct(row, "economic_need_index", "total_enrollment"), axis = 1)


# drop the districts that aren't geographic districts b/c we don't have test data for them
df = df[df["district"] < 33]

# (optional) get just the columns we need, to make it more manageable

cols = ['dbn',
        'year',
        'district',
        'boro',
        'school_name', 
        'total_enrollment',
        'female_1',
        'male_1',
        'asian_1', 
        'black_1', 
        'hispanic_1', 
        'multi_racial_1', 
        'native_american_1', 
        'white_1', 
        'students_with_disabilities_1', 
        'english_language_learners_1',  
        'poverty_1',
        'economic_need_index']

df = df[cols]
df

,dbn,year,district,boro,school_name,total_enrollment,female_1,male_1,asian_1,black_1,hispanic_1,multi_racial_1,native_american_1,white_1,students_with_disabilities_1,english_language_learners_1,poverty_1,economic_need_index
0,01M015,2016,1,Manhattan,P.S. 015 Roberto Clemente,178,0.466,0.534,0.079,0.287,0.590,0.017,0.006,0.022,0.287,0.067,0.854,0.882
1,01M015,2017,1,Manhattan,P.S. 015 Roberto Clemente,190,0.521,0.479,0.105,0.274,0.579,0.005,0.005,0.032,0.258,0.042,0.847,0.890
2,01M015,2018,1,Manhattan,P.S. 015 Roberto Clemente,174,0.489,0.511,0.138,0.276,0.546,0.000,0.006,0.034,0.224,0.046,0.845,0.888
3,01M015,2019,1,Manhattan,P.S. 015 Roberto Clemente,190,0.495,0.505,0.142,0.295,0.505,0.000,0.011,0.047,0.242,0.089,0.816,0.867
4,01M015,2020,1,Manhattan,P.S. 015 Roberto Clemente,193,0.523,0.477,0.135,0.275,0.528,0.005,0.000,0.057,0.223,0.109,0.819,0.856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7642,32K564,2016,32,Brooklyn,Bushwick Community High School,236,0.479,0.521,0.000,0.284,0.682,0.000,0.004,0.030,0.237,0.042,0.614,0.715
7643,32K564,2017,32,Brooklyn,Bushwick Community High School,263,0.479,0.521,0.000,0.308,0.673,0.000,0.000,0.019,0.323,0.072,0.859,0.907
7644,32K564,2018,32,Brooklyn,Bushwick Community High School,196,0.398,0.602,0.000,0.311,0.679,0.000,0.000,0.010,0.362,0.046,0.832,0.881
7645,32K564,2019,32,Brooklyn,Bushwick Community High School,214,0.416,0.584,0.000,0.229,0.766,0.000,0.000,0.005,0.327,0.065,0.883,0.904


In [38]:
# now load the math data and prep it a little bit
math_df = pd.read_csv(math_url)

# if the score isn't an int, set it to -1 to remove it or filter it easily
def clean_score(score):
    try:
        return int(score)
    except:
        return -1

math_df["mean_scale_score"] = math_df["mean_scale_score"].apply(clean_score)


display(math_df)

del math_df["school_name"]




,dbn,school_name,grade,year,category,number_tested,mean_scale_score,level1_n,level1_,level2_n,level2_,level3_n,level3_,level4_n,level4_,level3_4_n,level3_4_
0,01M015,PS 015 ROBERTO CLEMENTE,3,2013,All Students,27,278,16,59.3,11,40.7,0,0.0,0,0.0,0,0.0
1,01M015,PS 015 ROBERTO CLEMENTE,3,2014,All Students,18,286,6,33.3,9,50.0,2,11.1,1,5.6,3,16.7
2,01M015,PS 015 ROBERTO CLEMENTE,3,2015,All Students,17,280,10,58.8,4,23.5,2,11.8,1,5.9,3,17.6
3,01M015,PS 015 ROBERTO CLEMENTE,3,2016,All Students,21,275,13,61.9,4,19.0,4,19.0,0,0.0,4,19.0
4,01M015,PS 015 ROBERTO CLEMENTE,3,2017,All Students,29,302,8,27.6,9,31.0,7,24.1,5,17.2,12,41.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23891,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2013,All Students,145,266,107,73.8,35,24.1,3,2.1,0,0.0,3,2.1
23892,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2014,All Students,231,269,162,70.1,65,28.1,4,1.7,0,0.0,4,1.7
23893,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2015,All Students,324,268,249,76.9,70,21.6,4,1.2,1,0.3,5,1.5
23894,32K562,EVERGREEN MIDDLE SCHOOL FOR URBAN EXPLORATION,All Grades,2016,All Students,269,270,176,65.4,85,31.6,6,2.2,2,0.7,8,3.0


In [40]:
# give the cols better names that don't conflict with the ela data
col_map = {
  'number_tested': "number_tested_math",
  'mean_scale_score': "mean_scale_score_math",
  'level1_n': "level1_n_math",
  'level1_': "level1_math",
  'level2_n': "level2_n_math",
  'level2_': "level2_math",
  'level3_n': "level3_n_math",
  'level3_': "level3_math",
  'level4_n': "level4_n_math",
  'level4_': "level4_math",
  'level3_4_n': "level3_4_n_math",
  'level3_4_': "level3_4_math"
}

math_df = math_df.rename(columns=col_map)
print(df["year"].unique())
print(math_df["year"].unique())

[2016 2017 2018 2019 2020]
[2013 2014 2015 2016 2017]


In [41]:
# merge the math data into our main data frame (now it's only the demo data)
# API documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

                    # data        # matching rows only       # matching cols
combined = df.merge(math_df,      how="inner",               on=["dbn", "year"])
combined["year"].unique()



array([2016, 2017])

In [31]:
cols = ["dbn","school_name", "total_enrollment", "white_1", "black_1", "hispanic_1", "asian_1", "mean_scale_score_math"]

combined[cols]


,dbn,school_name,total_enrollment,white_1,black_1,hispanic_1,asian_1,mean_scale_score_math
0,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,275
1,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,304
2,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,283
3,01M015,P.S. 015 Roberto Clemente,178,0.022,0.287,0.590,0.079,286
4,01M015,P.S. 015 Roberto Clemente,190,0.032,0.274,0.579,0.105,302
...,...,...,...,...,...,...,...,...
9400,32K562,Evergreen Middle School for Urban Exploration,325,0.018,0.258,0.711,0.006,270
9401,32K562,Evergreen Middle School for Urban Exploration,342,0.026,0.249,0.722,0.003,265
9402,32K562,Evergreen Middle School for Urban Exploration,342,0.026,0.249,0.722,0.003,275
9403,32K562,Evergreen Middle School for Urban Exploration,342,0.026,0.249,0.722,0.003,268


In [35]:
# which schools had the highest mean math score in 2019
high_math = combined.query("grade == 'All Grades' and year == 2017 and white_1 < .15")

high_math = high_math.sort_values(by="mean_scale_score_math", ascending=False)

cols = ["dbn","school_name", "total_enrollment", "white_1", "black_1", "hispanic_1", "asian_1", "mean_scale_score_math"]
print("avg enrollment, top 100", high_math[cols][:100]["total_enrollment"].mean())

print("avg enrollment, bottom 100", high_math[cols][-100:]["total_enrollment"].mean())

high_math[cols]


avg enrollment, top 100 815.86
avg enrollment, bottom 100 397.35


,dbn,school_name,total_enrollment,white_1,black_1,hispanic_1,asian_1,mean_scale_score_math
4363,15K172,P.S. 172 Beacon School of Excellence,593,0.128,0.032,0.764,0.064,354
7315,26Q203,P.S. 203 Oakland Gardens,938,0.132,0.016,0.100,0.726,348
363,02M130,P.S. 130 Hernando De Soto,913,0.033,0.010,0.048,0.874,348
8097,28Q680,Queens Gateway to Health Sciences Secondary Sc...,690,0.042,0.271,0.097,0.506,346
7184,26Q046,P.S. 046 Alley Pond,556,0.137,0.045,0.156,0.628,346
...,...,...,...,...,...,...,...,...
1783,07X151,J.H.S. 151 Lou Gehrig,261,0.023,0.268,0.705,0.000,261
4400,15K429,Digital Arts and Cinema Technology High School,206,0.044,0.578,0.320,0.039,261
4185,14K586,Lyons Community School,496,0.026,0.371,0.593,0.010,259
7418,27Q053,M.S. 053 Brian Piccolo,267,0.015,0.521,0.431,0.011,258


In [56]:
# now load the math data and prep it a little bit
ela_df = pd.read_csv(ela_url)

ela_df["mean_scale_score"] = ela_df["mean_scale_score"].apply(clean_score)

del ela_df["school_name"]
del ela_df["unnamed_column"]
del ela_df["category"]

ela_df = ela_df.rename(columns={"mean_scale_score":"mean_scale_score_ela"})




/tmp/ipykernel_8281/796632093.py:2: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ela_df = pd.read_csv(ela_url)


In [57]:
# now combinethe ela stuff

ela_math_combo = combined.merge(ela_df, how="inner", on=["dbn", "year", "grade"])
ela_math_combo.columns


ela_math_combo.columns

Index(['dbn', 'year', 'district', 'boro', 'school_name', 'total_enrollment',
       'female_1', 'male_1', 'asian_1', 'black_1', 'hispanic_1',
       'multi_racial_1', 'native_american_1', 'white_1',
       'students_with_disabilities_1', 'english_language_learners_1',
       'poverty_1', 'economic_need_index', 'grade', 'category',
       'number_tested_math', 'mean_scale_score_math', 'level1_n_math',
       'level1_math', 'level2_n_math', 'level2_math', 'level3_n_math',
       'level3_math', 'level4_n_math', 'level4_math', 'level3_4_n_math',
       'level3_4_math', 'number_tested', 'mean_scale_score_ela', 'level_1',
       'level_1_1', 'level_2', 'level_2_1', 'level_3', 'level_3_1', 'level_4',
       'level_4_1', 'level_3_4', 'level_3_4_1'],
      dtype='object')

In [61]:
# which schools had the highest mean math score in 2019
high_math = ela_math_combo.query("grade == 'All Grades' and year == 2017 and white_1 < .15")

high_math = high_math.sort_values(by="mean_scale_score_math", ascending=False)

cols = ["dbn","school_name", "total_enrollment", "white_1", "black_1", "hispanic_1", "asian_1", "avg_score"]



high_math["avg_score"] = (high_math["mean_scale_score_math"] + high_math["mean_scale_score_ela"]) / 2

high_math[cols][:10]

,dbn,school_name,total_enrollment,white_1,black_1,hispanic_1,asian_1,avg_score
4352,15K172,P.S. 172 Beacon School of Excellence,593,0.128,0.032,0.764,0.064,350.0
360,02M130,P.S. 130 Hernando De Soto,913,0.033,0.010,0.048,0.874,341.5
7299,26Q203,P.S. 203 Oakland Gardens,938,0.132,0.016,0.100,0.726,344.5
8081,28Q680,Queens Gateway to Health Sciences Secondary Sc...,690,0.042,0.271,0.097,0.506,345.0
7168,26Q046,P.S. 046 Alley Pond,556,0.137,0.045,0.156,0.628,337.0
7072,25Q242,P.S. 242 Leonard P. Stavisky Early Childhood S...,392,0.046,0.036,0.148,0.763,335.5
9380,32K554,All City Leadership Secondary School,435,0.080,0.083,0.763,0.071,340.0
267,02M042,P.S. 042 Benjamin Altman,653,0.025,0.014,0.067,0.887,331.0
1867,07X359,Concourse Village Elementary School,356,0.022,0.323,0.635,0.008,338.0
105,01M184,P.S. 184m Shuang Wen,684,0.073,0.037,0.135,0.694,332.0
